<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/llama_2_13b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip -q install bitsandbytes accelerate

In [ ]:
!pip show bitsandbytes

In [3]:
from transformers import BitsAndBytesConfig

In [4]:
model_id = "chenshake/Llama-2-13b-chat-hf"

In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [8]:
%%time
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=quantization_config,
                                             device_map="auto")

config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00006.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

pytorch_model-00003-of-00006.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

pytorch_model-00005-of-00006.bin:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

CPU times: user 1min 6s, sys: 1min 18s, total: 2min 25s
Wall time: 10min 52s


In [9]:
prompt = 'I liked "Yesterday Once More". Do you have any recommendations of other song I might like?\n'

In [10]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")

In [11]:
input_ids

tensor([[    1,   306, 23289,   376, 29979, 18358,  9038,  5853,  1642,  1938,
           366,   505,   738,  6907,   800,   310,   916,  4823,   306,  1795,
           763, 29973,    13]])

In [12]:
%%time
with torch.no_grad():
  output = model.generate(input_ids=input_ids,
                          max_new_tokens=2048,
                          temperature=0.6)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') befor

CPU times: user 36.8 s, sys: 1.67 s, total: 38.5 s
Wall time: 41 s


In [13]:
%%time
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

I liked "Yesterday Once More". Do you have any recommendations of other song I might like?

I'm glad you enjoyed "Yesterday Once More"! If you're looking for more songs that are similar in style or mood, here are a few recommendations:

1. "The Way We Were" by Barbra Streisand - This is a classic ballad that, like "Yesterday Once More," has a nostalgic and introspective feel.
2. "I Will Always Love You" by Whitney Houston - This is a powerful love ballad that features a soaring vocal performance, similar to the one delivered by Karen Carpenter in "Yesterday Once More."
3. "Can't Help Falling in Love" by Elvis Presley - This is a romantic ballad that has a similar feel to "Yesterday Once More" in terms of its mellow, introspective mood.
4. "Unchained Melody" by The Righteous Brothers - This is another classic love ballad that features a soaring vocal performance and a nostalgic, yearning quality that is similar to "Yesterday Once More."
5. "A Song for You" by Donny Hathaway - This is a 

In [14]:
%%time
prompt = "Tell me about AI"

input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=2048)
print(tokenizer.decode(output[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Tell me about AI in the workplace

Artificial intelligence (AI) is transforming the workplace in a number of ways, from automating routine tasks to enhancing decision-making processes. Here are some key trends and insights about AI in the workplace:

1. Automation: AI is being used to automate routine tasks such as data entry, scheduling, and customer service. This frees up employees to focus on more complex and creative work.
2. Decision-making: AI is being used to analyze large amounts of data and make decisions based on patterns and trends. This can help businesses make more informed decisions and improve operational efficiency.
3. Personalization: AI is being used to personalize customer experiences, such as personalized product recommendations and personalized content.
4. Virtual assistants: AI-powered virtual assistants are being used to help employees with tasks such as scheduling meetings, setting reminders, and answering questions.
5. Predictive maintenance: AI is being us

In [ ]:
%%time
prompt = "Tell me about Facebook boss"

input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=2048)
print(tokenizer.decode(output[0]))

In [ ]:
%%time
pipeline = transformers.pipeline(
                                  "text-generation",
                                  model=model_id,
                                  device_map="cuda:0"
                                )